# Input widgets

To interact with the [HERE search & geocoding API](https://developer.here.com/documentation/geocoding-search-api/api-reference-swagger.html) services `here_search` provides a few widgets based on [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/index.html):
- `SubmittableTextBox` and `TermsButtons` to formulate textual search intents for Autosuggest and Discover
- `PlaceTaxonomyButtons` to formulate custom place classification shortcuts for Browse
- `OneBoxSimple` to associate search intents to search requests

Those widgets are using a queue to exchange objects of `SearchIntent` class derivatives.

### SubmittableTextBox

`SubmittableTextBox` is a special `Text` widget supporting key strokes events and text submissions.<br/>
It uses a queue to post `SearchIntent` objects, either `TransientIntent` for each key stroke or `FornulatedIntent` when the transient text is submitted. 

In the snippet below, key strokes and text submissions are displayed in an `Output` widget.

In [ ]:
from here_search.widgets.input import SubmittableTextBox
from here_search.widgets.util import Output
from here_search.entities import FormulatedIntent
from asyncio import ensure_future, Queue

box = SubmittableTextBox(Queue())
out_keys, out_text = Output(height=30), Output(height=30)

async def print_text():
    while True:
        intent = await box.queue.get()
        out = out_text if isinstance(intent, FormulatedIntent) else out_keys
        with out:
            out.replace(intent.materialization)

t1 = ensure_future(print_text())
display(box, out_keys, out_text)

### TermsButtons

`TermsButtons` objects are taking a `SubmittableTextBox` instance to feed it with predefined values when the buttons are selected.<br/>
Per default, the complete form value is overwritten by the button text. 

The constructor `index` parameter allows to specify wich token is overwritten:

In [ ]:
from here_search.widgets.input import TermsButtons

box2 = SubmittableTextBox(Queue())
out_keys2 = Output(height=30)

async def print_text2():
    while True:
        intent = await box2.queue.get()
        with out_keys2:
            out_keys2.replace(intent.materialization)
            
terms = TermsButtons(box2, 
                     values=["text 1", "text 2"], 
                     index=-1)

t2 = ensure_future(print_text2())
display(box2, terms, out_keys2)

### PlaceTaxonomyButtons

`PlaceTaxonomyButtons` is a widget associating a list of buttons to a place classification.<br/>
It uses a queue to post a `FormulatedIntent` object materialized with the place taxonomy item when a button is selected.

The example belows uses the `PlaceTaxonomyExample` object. Each buttons selection displays the related taxonomy item in an `Output` widget.

(fontawesome 5.12.0 [download link](https://use.fontawesome.com/releases/v5.12.0/fontawesome-free-5.12.0-web.zip))

In [ ]:
from here_search.entities import PlaceTaxonomyExample
from here_search.widgets.input import PlaceTaxonomyButtons

buttons = PlaceTaxonomyButtons(Queue(), PlaceTaxonomyExample.taxonomy, PlaceTaxonomyExample.icons)
out_onto = Output(height=30)

async def print_taxonomy():
    while True:
        with out_onto:
            intent = await buttons.queue.get()
            out_onto.replace(str(intent.materialization))
            
t4 = ensure_future(print_taxonomy())
display(buttons, out_onto)

## ResponseMap

Search end-users fire requests (text queries, or ontology shortcuts) from a particular location context, materialized in the API through the search center lat/lon. It is convenient to capture a search center thrioygh the use of a draggable map.

The `ResponseMap` class uses [`here-map-widget`](https://pypi.org/project/here-map-widget-for-jupyter/) `Map` class dragging and zooming capabilities to capture the center of a HERE map and sends it to an observer function. This class expects a `position_handler` constructor parameter respecting the traitlets event handler [signature](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Registering-callbacks-to-trait-changes-in-the-kernel). In the snippet below, the map position is only displayed in an `Output` widget, each time the user interacts with the map:

In [ ]:
from here_search.widgets.output import ResponseMap
from here_search.api import API

out_map = Output(height=30)

def position_handler(latitude, longitude):
    with out_map:
        out_map.replace(f'{latitude},{longitude}')
    
rmap = ResponseMap(api_key=API().api_key, 
                   center=(52.51604, 13.37691),
                   layout = {'height': '200px'},
                   position_handler=position_handler)

display(rmap, out_map)

## OneBoxSimple

`OneBoxSimple` brings the previously described widgets together to offer both a one-box and a taxonomy search experience.
For this, a asyncio task is continusouly waiting for search events, and dispatch them to the dedicated search request and response handlers:

- `FormulatedIntent` from `SubmittableTextBox` are sent to Discover. Response items are displayed in a clickable result list, on a `ResponseMap` and in JSON Output.
- Other `SearchIntent` are sent to Autosuggest

In the snippet below, `App` uses this class together with
- `SubmittableTextBox`, `TermsButtons` and `PlaceTaxonomyButtons` widgets to feed the three event types of `OneBoxSimple`
- a `Output` widget to handle the API responses
- a `ResponseMap` widget to 
  - capture the search center from search results and user interactions
  - display location response items

In [ ]:
from here_search.base import OneBoxSimple
from json import dumps

queue4 = Queue()
text_box = SubmittableTextBox(queue4)
terms_buttons = TermsButtons(text_box, index=-1)
taxonomy_buttons = PlaceTaxonomyButtons(queue4, PlaceTaxonomyExample.taxonomy, PlaceTaxonomyExample.icons)
rmap = ResponseMap(api_key=API().api_key, center=(52.51604, 13.37691), layout = {'height': '200px'})
out_simple = Output(height=160)

class App(OneBoxSimple):
    default_language = "de"
    
    def __init__(self, queue):
        super().__init__(queue=queue)
        self.text_box = text_box
        self.terms_buttons = terms_buttons
        self.taxonomy_buttons = taxonomy_buttons
        self.out = out_simple
        self.rmap = rmap
        def handler(latitude, longitude):
            self.search_center = latitude, longitude
        self.rmap.set_position_handler(handler)
    
    def handle_suggestion_list(self, resp, session):
        with self.out:
            self.out.replace(dumps(resp.data, indent=2))
        self.terms_buttons.set(resp.terms)

    def handle_empty_text_submission(self, session):
        with self.out:
            self.out.clear_output()
        self.terms_buttons.set([])
        
    def handle_result_list(self, resp, session):
        with self.out:
            self.out.replace(dumps(resp.data, indent=2))
        self.text_box.text_w.value = ''
        self.terms_buttons.set([])
        self.rmap.display(resp)

app = App(queue4).run()
display(taxonomy_buttons, text_box, terms_buttons, rmap, out_simple)

# OneBoxMap


`OneBoxMap` is a demo application showing how [HERE Geocoding and Search](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics/endpoint-autosuggest-brief.html) `/autosuggest`, `/discover` and `/lookup` endpoints are meant to be used:
- The app  
    - proposes a single text form to formulate queries
    - displays three buttons with predicted last token text completions
    - displays all API calls in a separate window
    - displays search results on a map and a list
<br/>
- `/autosuggest` endpoint is called for each key-stroke.
    - Follow-up query suggestions are displayed in the list only
    - Location suggestions are displayed in the list and on the map
    - Query term suggestions are displayed in three buttons
    - Selected locations lead to a call to `/lookup` using the location record `id`.
    - Selected query suggestions lead to a http GET using the query `href` value
<br/>
- `/discover` endpoint is called when a query submission is validated.
    - Validation happens when the [return] key or the <img src="https://upload.wikimedia.org/wikipedia/commons/2/2b/Font_Awesome_5_solid_search.svg" style="width:12px"/> button are hit
    - Location results are displayed in the list and on the map
    - Selected results lead to a call to `/lookup` using the location record `id`.
<br/>
- `/autosuggest` and `/discover` requests are sent using the map center as search center
- `/browse` endpoint is called for each tanonomy button selected.
- Search requests are cached during the lifetime of the application


<!-- https://commons.wikimedia.org/wiki/File:Font_Awesome_5_solid_search.svg -->

In [ ]:
from here_search.widgets.app import OneBoxMap
OneBoxMap().run()